# Face Embedding

* It is a vector that represents the features extracted from the face. It can then be compared with vectors generated for other faces. To better explain, for example, another vector that is close may be the same person, whereas another vector that is far may be a different person.

* This classifier model will take a face embedding as input and predict the identity of the face.

* The FaceNet model can be used as part of the classifier itself, or it can be used to pre-process a face to create a face embedding that can be stored and used as input to the classifier model. The latter approach is preferred as the FaceNet model is both large and slow to create a face embedding.

First we can load our dataset using the load()

In [1]:
from numpy import load
data = load('compressed_data.npz')

In [2]:
print(data)

In [3]:
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded:\n',
      'trainX: ', trainX.shape ,'\n',
      'trainy: ', trainy.shape ,'\n',
      'testX: ', testX.shape ,'\n',
      'testy: ', testy.shape ,'\n',
    )

Loaded:
 trainX:  (92, 160, 160, 3) 
 trainy:  (92,) 
 testX:  (25, 160, 160, 3) 
 testy:  (25,) 



In [4]:
#load the model
from keras.models import load_model
model = load_model('model/facenet_keras.h5')
print('Loaded Model!!')

Using TensorFlow backend.
W0722 13:52:21.230991 4626490816 deprecation_wrapper.py:119] From /Users/pietraferreira/.virtualenvs/face_recognition_2207/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 13:52:21.328984 4626490816 deprecation_wrapper.py:119] From /Users/pietraferreira/.virtualenvs/face_recognition_2207/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 13:52:21.352975 4626490816 deprecation_wrapper.py:119] From /Users/pietraferreira/.virtualenvs/face_recognition_2207/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 13:52:21.353719 4626490816 deprecation_wrapper.py:119] From /Users/pietraferreira/.virtualenvs/face_recognition_2207/lib/python3

Loaded Model!!


/Users/pietraferreira/.virtualenvs/face_recognition_2207/lib/python3.7/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


We can then enumerate each face in the train and test dataset to predict an embedding.

To predict an embedding, the pixel values of the image need to be suitably prepared to meet the expectations of the model. This implementation of the FaceNet model expects that the pixel values are **standardized**.

The script would look like this:

In [5]:
from numpy import expand_dims
#get the face embedding for one face
def get_embedding(model, face_pixels):
    #scale pixel values
    face_pixels = face_pixels.astype('float32')
    #standardize pixel values across channels
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    #transform face into one single sample
    samples = expand_dims(face_pixels, axis=0)
    #make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

Now tying everything together and loading everything:

In [6]:
from numpy import asarray
#convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

KeyboardInterrupt: 

In [ ]:
#same thing for the test set
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

In [ ]:
from numpy import savez_compressed
#save arrays to one file, compressed
savez_compressed('compressed_face-embeddings.npz', newTrainX, trainy, newTestX, testy)

Now we transformed the train dataset into 92 face embeddings, each comprised of a 128 element vector.

The 25 examples in the test dataset were also converted to face embeddings of 128.